In [24]:
import math
import random
import numpy as np
# Define the length of the DNA reads
read_length = 150

# Define the number of reads in the dataset
num_reads = 1000000

# Generate random DNA sequences for dataset1 and dataset2
def generate_reads(num_reads, read_length):
    bases = ['A', 'C', 'G', 'T']
    reads = []
    for i in range(num_reads):
        read = ''.join([random.choice(bases) for _ in range(read_length)])
        reads.append(read)
    return reads

dataset1_reads = generate_reads(num_reads, read_length)
dataset2_reads = generate_reads(num_reads, read_length)

# Convert DNA sequences to numerical representation
def encode_reads(reads):
    encoding = {'A': [1,0,0,0], 'C': [0,1,0,0], 'G': [0,0,1,0], 'T': [0,0,0,1]}
    encoded_reads = []
    for read in reads:
        encoded_read = [encoding[base] for base in read]
        encoded_reads.append(np.concatenate(encoded_read))
    return np.array(encoded_reads)

dataset1_encoded_reads = encode_reads(dataset1_reads)
dataset2_encoded_reads = encode_reads(dataset2_reads)

# Generate labels for pairs of reads based on overlap length
def generate_labels(reads1, reads2):
    num_reads = len(reads1)
    labels = np.zeros(num_reads)
    for i in range(num_reads):
        overlap_length = 0
        for j in range(read_length):
            if reads1[i][-j:] == reads2[i][:j]:
                overlap_length = j
        labels[i] = math.exp(overlap_length / read_length)
    return labels

split_size = int(num_reads * 0.8)
train_similarities = generate_labels(dataset1_reads[:split_size], dataset2_reads[:split_size])
val_similarities = generate_labels(dataset1_reads[split_size:], dataset2_reads[split_size:])


# Split dataset1 and dataset2 into training and validation sets
train_size = int(0.8 * num_reads)
indices = np.random.permutation(num_reads)
train_indices = indices[:train_size]
val_indices = indices[train_size:]

train_dataset1_reads = dataset1_encoded_reads[train_indices]
val_dataset1_reads = dataset1_encoded_reads[val_indices]
train_dataset2_reads = dataset2_encoded_reads[train_indices]
val_dataset2_reads = dataset2_encoded_reads[val_indices]


In [26]:
print(dataset1_reads[2]) 
print(dataset2_reads[2])
print(train_similarities[2])

TGGGGAGAGGGGTTTTCATGTACGTGGGATGTTTAACTTGCGGAATGATTTACGCCCGGTTTAGGAGGTTAGACTATACGCGTAGCCCAAGTATACAGCCCAGCATTAGCAAGCTACGGTCCATATACCAACTCGGACCAGGTTTACCCG
GGTAGGCAATGCGCCCATGGTGCCAATCTCCGCTACTGGGCTTCATGCGATAGGGAAATGAAACCTCAGGACCAAGAGTATTGCTTGTCTGATTGGTCTACAACGTGGATGTCCTATCGGGGTCGCCTTAGTACTTCAGGGGGAGGAAGC
1.0066889383540194


In [27]:
read1 = "AACTGGTACGTCAGTTACGCTAGGTTCAAGTGGCTAGCTACGTCGATAGCTGTCAGGCTAGCTAGGAGCTAGCTGACTGATCGATCGATCGATCGGATCGATCTACGATCGATCGATCGACGTCAGCTGACTGATCGATCGATCGATCGGATCGATC"
read2 = "CTGATCGATCGATCGATCGGATCGATCTACGATCGATCGATCGACGTCAGCTGACTGATCGATCGATCGATCGGATCGATCTGACGTACGATCGATCGATCGATCGGATCGATCAGCTAGCTAGCTAAGGCTAGCTAGCTAGGAGCTAGCTGACTGATC"

generate_labels([read1], [read2])

array([1.71600686])

In [28]:
import torch
import torch.nn as nn
import torch.nn.functional as F

# Define the size of the hidden layer
hidden_size = 64

# Define the input shape for the DNA reads
input_shape = (4, read_length)

# Define the siamese autoencoder model
class SiameseAutoencoder(nn.Module):
    def __init__(self):
        super(SiameseAutoencoder, self).__init__()
        
        # Define the encoder layers for dataset1 and dataset2
        self.encoder = nn.Sequential(
            nn.Linear(input_shape[0]*input_shape[1], hidden_size),
            nn.ReLU(),
        )
        
        # Define the decoder layers for dataset1 and dataset2
        self.decoder = nn.Sequential(
            nn.Linear(hidden_size, input_shape[0]*input_shape[1]),
            nn.Sigmoid(),
        )

    def forward(self, input1, input2):
        # Compute the embeddings for dataset1 and dataset2
        embedding1 = self.encoder(input1.view(-1, input_shape[0]*input_shape[1]))
        embedding2 = self.encoder(input2.view(-1, input_shape[0]*input_shape[1]))
        
        # Compute the distance between the embeddings
        distance = torch.norm(embedding1 - embedding2, p=2, dim=1, keepdim=True)
        
        # Reconstruct the DNA reads from the embeddings
        reconstruction1 = self.decoder(embedding1)
        reconstruction2 = self.decoder(embedding2)
        
        # Concatenate the reconstructions and distance
        output = torch.cat((reconstruction1, reconstruction2, distance), dim=1)
        
        return output

# Define the siamese autoencoder model
model = SiameseAutoencoder()

# Define the loss function
def contrastive_loss(output, target):
    margin = 1.0
    distances = output[:, -1].unsqueeze(1)
    reconstructions1 = output[:, :input_shape[0]*input_shape[1]].view(-1, input_shape[0], input_shape[1])
    reconstructions2 = output[:, input_shape[0]*input_shape[1]:-1].view(-1, input_shape[0], input_shape[1])
    distance_similarity = torch.exp(-distances)
    loss = 0.5 * ((1-target) * F.mse_loss(reconstructions1, reconstructions2) + target * torch.pow(distances - margin, 2) * distance_similarity)
    return loss.mean()

# Define the optimizer
optimizer = torch.optim.Adam(model.parameters(), lr=1e-3)


In [29]:
import torch.utils.data as data

class DNAReadsDataset(data.Dataset):
    def __init__(self, reads1, reads2, similarities):
        self.reads1 = reads1
        self.reads2 = reads2
        self.similarities = similarities
        
    def __getitem__(self, index):
        input1 = torch.Tensor(one_hot_encode(self.reads1[index]))
        input2 = torch.Tensor(one_hot_encode(self.reads2[index]))
        target = torch.Tensor([self.similarities[index]])
        return input1, input2, target
    
    def __len__(self):
        return len(self.reads1)
    
def one_hot_encode(read):
    encoding = np.zeros((4, len(read)))
    for i, base in enumerate(read):
        if base == 'A':
            encoding[0, i] = 1
        elif base == 'C':
            encoding[1, i] = 1
        elif base == 'G':
            encoding[2, i] = 1
        elif base == 'T':
            encoding[3, i] = 1
    return encoding



In [30]:
# Define the number of epochs and batch size
num_epochs = 20000
batch_size = 32

# Define the data loaders for the training and validation sets
train_dataset = DNAReadsDataset(dataset1_reads[:split_size], dataset2_reads[:split_size], train_similarities)
train_loader = torch.utils.data.DataLoader(train_dataset, batch_size=batch_size, shuffle=True)

val_dataset = DNAReadsDataset(dataset1_reads[split_size:], dataset2_reads[split_size:], val_similarities)
val_loader = torch.utils.data.DataLoader(val_dataset, batch_size=batch_size, shuffle=False)

# Train the siamese autoencoder model
for epoch in range(num_epochs):
    # Train the model on the training set
    model.train()
    train_loss = 0
    for batch_idx, (input1, input2, target) in enumerate(train_loader):
        optimizer.zero_grad()
        output = model(input1, input2)
        loss = contrastive_loss(output, target)
        loss.backward()
        optimizer.step()
        train_loss += loss.item()
    train_loss /= len(train_loader)
    
    # Evaluate the model on the validation set
    model.eval()
    val_loss = 0
    with torch.no_grad():
        for batch_idx, (input1, input2, target) in enumerate(val_loader):
            output = model(input1, input2)
            loss = contrastive_loss(output, target)
            val_loss += loss.item()
    val_loss /= len(val_loader)
    
    # Print the loss for this epoch
    print('Epoch [{}/{}], Train Loss: {:.4f}, Val Loss: {:.4f}'.format(epoch+1, num_epochs, train_loss, val_loss))


Epoch [1/20000], Train Loss: 0.0068, Val Loss: 0.0069
Epoch [2/20000], Train Loss: 0.0066, Val Loss: 0.0065
Epoch [3/20000], Train Loss: 0.0066, Val Loss: 0.0067
Epoch [4/20000], Train Loss: 0.0066, Val Loss: 0.0065
Epoch [5/20000], Train Loss: 0.0066, Val Loss: 0.0064
Epoch [6/20000], Train Loss: 0.0066, Val Loss: 0.0065
Epoch [7/20000], Train Loss: 0.0066, Val Loss: 0.0070
Epoch [8/20000], Train Loss: 0.0065, Val Loss: 0.0063
Epoch [9/20000], Train Loss: 0.0065, Val Loss: 0.0062
Epoch [10/20000], Train Loss: 0.0066, Val Loss: 0.0065
Epoch [11/20000], Train Loss: 0.0066, Val Loss: 0.0066
Epoch [12/20000], Train Loss: 0.0065, Val Loss: 0.0064
Epoch [13/20000], Train Loss: 0.0065, Val Loss: 0.0063
Epoch [14/20000], Train Loss: 0.0065, Val Loss: 0.0065
Epoch [15/20000], Train Loss: 0.0065, Val Loss: 0.0064
Epoch [16/20000], Train Loss: 0.0065, Val Loss: 0.0061
Epoch [17/20000], Train Loss: 0.0065, Val Loss: 0.0065
Epoch [18/20000], Train Loss: 0.0065, Val Loss: 0.0069
Epoch [19/20000], T